# 소형주 + 저PBR 전략

## Basic Setting

In [7]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib import request as rq
from tqdm import tqdm

In [8]:
sample_code = '005930'
SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'

## Get Market Value

In [55]:
snap_url = rq.urlopen(SNAP_URL.format(sample_code)).read()
snap_soup = BeautifulSoup(snap_url, 'html.parser')

In [56]:
mv_cells = snap_soup.find('div', {'id':'svdMainGrid1'}).find_all('td')
market_value = float(mv_cells[6].string.replace(',',''))

## Get PBR

In [23]:
pbr_cells = snap_soup.find('div', {'id':'corp_group2'}).find_all('dd')
pbr = float(pbr_cells[7].string.replace(',',''))

## Get All Stock Code

In [25]:
company = pd.read_csv('company.csv')

In [42]:
code_list = company['종목코드'].dropna()

In [43]:
code_list.head()

0    000155
1    00088K
2    010955
3    051915
4    071055
Name: 종목코드, dtype: object

## Create Crawling Function

In [57]:
def crawler(li):
    # dict for DataFrame
    result = {}
    
    for code in tqdm(li):
        try:
            # get company name
            name = company[company['종목코드'] == code]['회사명'].values[0]
            
            # set URL
            SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
            
            snap_url = rq.urlopen(SNAP_URL.format(code)).read()
            snap_soup = BeautifulSoup(snap_url, 'html.parser')
            
            # get Market Value
            mv_cells = snap_soup.find('div', {'id':'svdMainGrid1'}).find_all('td')
            market_value = float(mv_cells[6].string.replace(',',''))
            
            # get PBR
            pbr_cells = snap_soup.find('div', {'id':'corp_group2'}).find_all('dd')
            pbr = float(pbr_cells[7].string.replace(',',''))
            
            result[name] = [code, market_value, pbr]
        except (TypeError, IndexError, AttributeError, ValueError):
            
            pass
        
    # convert to DataFrame
    result = pd.DataFrame(result)
    
    # result transposing
    result = result.transpose()
    
    # set column name
    column_names = ['Code', 'Market_Value', 'PBR']
    result.columns = column_names
    
    return result

In [58]:
sample_df = crawler(code_list)

 12%|█▏        | 240/2021 [01:51<13:50,  2.15it/s]


KeyboardInterrupt: 

In [60]:
copy_df = sample_df.copy()

## Screening

In [69]:
copy_df = copy_df[copy_df.index.str.contains('스팩|SPAC')]

In [75]:
final_result = copy_df[copy_df.PBR > 0.2]
final_result

,Code,Market_Value,PBR
엔에이치스팩12호,273060,850.0,2.2
한국제5호스팩,271740,82.0,1.1
하나금융9호스팩,261200,1266.0,3.93
교보7호스팩,267320,1280.0,4.42
한화수성스팩,265920,91.0,1.07
IBKS제6호스팩,264850,1026.0,3.24
미래에셋대우스팩1호,265480,99.0,1.12
엔에이치스팩10호,256630,1703.0,2.17
한화에이스스팩3호,264290,63.0,1.08
케이비제11호스팩,258790,620.0,3.43


In [76]:
small_range = int(len(final_result) * 0.2)
final_result = final_result.sort_values(by='Market_Value')[:small_range]

In [77]:
final_result

,Code,Market_Value,PBR
엔에이치SL스팩,207720,51.0,1.13
한화에이스스팩3호,264290,63.0,1.08
신한제3호스팩,257730,72.0,1.11
SK3호스팩,232330,74.0,1.08
골든브릿지제2호스팩,206660,80.0,1.07
한국제5호스팩,271740,82.0,1.1
이베스트스팩3호,225440,84.0,1.02
한화수성스팩,265920,91.0,1.07
엔에이치스팩7호,217810,98.0,1.08
미래에셋대우스팩1호,265480,99.0,1.12


In [80]:
final_result.sort_values(by='PBR').head(30)

,Code,Market_Value,PBR
이베스트스팩3호,225440,84.0,1.02
골든브릿지제3호스팩,219580,99.0,1.06
골든브릿지제2호스팩,206660,80.0,1.07
한화수성스팩,265920,91.0,1.07
한화에이스스팩3호,264290,63.0,1.08
SK3호스팩,232330,74.0,1.08
엔에이치스팩7호,217810,98.0,1.08
한국제5호스팩,271740,82.0,1.1
신한제3호스팩,257730,72.0,1.11
미래에셋대우스팩1호,265480,99.0,1.12
